In [ ]:
import numpy as np

In [ ]:
class PCA_:
    def __init__(self, x_data, cutoff=0.95, kernel=None, degree=3, n_greater=False):
        self.data = x_data
        self.data_size, self.dims = x_data.shape
        self.cutoff = cutoff
        self.kernel = kernel
        self.degree = degree
        self.n_greater = n_greater

        self.top_eigen_pairs = None
        self.mean = None

        self.eigen_vector_pairs = None 
        self.cutoff_point = 0 

    def _center_data(self):
        self.mean = np.mean(self.data, axis=0)
        return self.data - self.mean
    
    def _center_kernel(self,kernel): 
        n = kernel.shape[0]
        one_n = np.ones((n,n))/n 
        k_centered = kernel - one_n @kernel - kernel@one_n + one_n@one_n
        return k_centered
     
    def _kernel_transform(self, X):
        if self.kernel is None and self.n_greater:
            return np.dot(X , X.T) #already centerd due to centering of the input data
        elif self.kernel is None and self.n_greater is False: 
            return np.dot(X.T , X)
        elif self.kernel == "poly":
            return self._center_kernel((np.dot(X, X.T) + 1) ** self.degree)
        elif self.kernel == "rbf":
            gamma = 1 / X.shape[1]
            sq_dists = np.sum(X ** 2, axis=1).reshape(-1, 1) + \
                       np.sum(X ** 2, axis=1) - 2 * np.dot(X, X.T)
            return self._center_kernel(np.exp(-gamma * sq_dists))
        
    def _cutoff(self, sorted_eigen_values):
        total = np.sum(sorted_eigen_values)
        accum = 0
        for j in range(len(sorted_eigen_values)):
            accum += sorted_eigen_values[j]
            if accum / total >= self.cutoff:
                return j + 1  # +1 to include this component
        return len(sorted_eigen_values)  # fallback in case total not reached
        
    def train(self):
        X = self._center_data() # centering dataset 
        centered_kernel  = self._kernel_transform(X) # returns the centered_kernel for the dataset (if kernel = "None" returns the covarience matrix)

        # till now i am working with (which can lead to large time complexity)

        if(self.kernel is None and self.n_greater is False):
            # do straight up calculation of the covarience matrix and store the eigen directions and the cutoff point

            eigen_vals, eigen_vecs = np.linalg.eigh(self.kernel)
            sorted_idx = np.argsort(eigen_vals)[::-1]

            eigen_vals = eigen_vals[sorted_idx]
            eigen_vecs = eigen_vecs[:, sorted_idx]

            self.cutoff_point = self._cutoff(eigen_vals)
            
            top_vals = eigen_vals[:self.cutoff_point]
            top_vecs = eigen_vecs[:, :self.cutoff_point]

            # Normalize eigenvectors (each column vector to unit norm)
            norms = np.linalg.norm(top_vecs, axis=0, keepdims=True)
            top_vecs_normalized = top_vecs / norms

            self.components = top_vecs_normalized
            pass
        elif (self.kernel is None and self.n_greater is True): 
            # find the eigen values and directions to get the coefficient and then find the eigen directions with that 
            eigen_vals, eigen_vecs = np.linalg.eigh(self.kernel)
            sorted_idx = np.argsort(eigen_vals)[::-1]

            eigen_vals = eigen_vals[sorted_idx]
            eigen_vecs = eigen_vecs[:, sorted_idx]

            # Normalize eigenvectors of G by sqrt of eigenvalues
            eigen_vecs_norm = eigen_vecs / np.sqrt(eigen_vals + 1e-10)

            # Convert coefficients to eigen directions in original space
            eigen_dirs = X.T @ eigen_vecs_norm  # d x n

            # Now apply cutoff on eigenvalues and corresponding directions
            self.cutoff_point = self._cutoff(eigen_vals)
            top_vals = eigen_vals[:self.cutoff_point]
            top_dirs = eigen_dirs[:, :self.cutoff_point]

            # Normalize eigen directions to unit length (optional but recommended)
            norms = np.linalg.norm(top_dirs, axis=0, keepdims=True)
            self.components = top_dirs / norms

            # Save info for transform
            self.mean = np.mean(X, axis=0)
            pass
        else: 
            # store all the datapoints and then when a new datapoint comes in find the sum(coefficient*kernel(datapoint , xi))
            eigen_vals, eigen_vecs = np.linalg.eigh(self.kernel)
            sorted_idx = np.argsort(eigen_vals)[::-1]

            eigen_vals = eigen_vals[sorted_idx]
            eigen_vecs = eigen_vecs[:, sorted_idx]

            self.cutoff_point = self._cutoff(eigen_vals)

            top_vals = eigen_vals[:self.cutoff_point]
            # Normalize top eigenvectors directly here
            top_vecs = eigen_vecs[:, :self.cutoff_point] / np.sqrt(top_vals + 1e-10)

            self.alphas = top_vecs
            self.X_fit = X
            pass

    def transform(self, X_new):
        if self.kernel is None and self.n_greater is False:
            # Linear PCA
            X_centered = X_new - np.mean(self.X_fit, axis=0)
            return X_centered @ self.components

        elif self.kernel is None and self.n_greater is True:
            # Dual PCA
            X_centered = X_new - self.mean
            return X_centered @ self.components

        else:
            # Kernel PCA
            # Compute kernel matrix between X_new and training samples
            K_new = np.array([[self.kernel(x_new, x_train) for x_train in self.X_fit] for x_new in X_new])
            return K_new @ self.alphas

